# Soft Computing

## Vežba 1 - Digitalna slika, computer vision, OpenCV

### OpenCV

Open source biblioteka namenjena oblasti računarske vizije (eng. *computer vision*). Dokumentacija dostupna  [ovde](https://opencv.org/).

### matplotlib

Plotting biblioteka za programski jezik Python i njegov numerički paket NumPy. Dokumentacija dostupna [ovde](https://matplotlib.org/).

### Učitavanje slike

OpenCV metoda za učitavanje slike sa diska je <b>imread(path_to_image)</b>, koja kao parametar prima putanju do slike na disku. Učitana slika <i>img</i> je zapravo NumPy matrica, čije dimenzije zavise od same prirode slike. Ako je slika u boji, onda je <i>img</i> trodimenzionalna matrica, čije su prve dve dimenzije visina i širina slike, a treća dimenzija je veličine 3, zato što ona predstavlja boju (RGB, po jedan segment za svaku osnonvu boju).

In [ ]:
import numpy as np
import cv2 # OpenCV biblioteka
import matplotlib
import matplotlib.pyplot as plt  

In [ ]:
# iscrtavanje slika i grafika unutar samog browsera
%matplotlib inline 
# prikaz vecih slika 
matplotlib.rcParams['figure.figsize'] = 16,12

In [ ]:
img = cv2.imread('images/girl.jpg') # ucitavanje slike sa diska
plt.imshow(img)  # prikazivanje slike

In [ ]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # konvertovanje iz BGR u RGB model boja (OpenCV ucita sliku kao BGR)
plt.imshow(img)

In [ ]:
img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) # konvertovanje iz RGB u HSV model boja
plt.imshow(img_hsv)

### Prikazivanje dimenzija slike

In [ ]:
print(img.shape)  # shape je property Numpy array-a za prikaz dimenzija
print("Visina slike: ", img.shape[0])
print("Sirina slike: ", img.shape[1])
print("Broj komponenti: ", img.shape[2])

Obratiti pažnju da slika u boji ima 3 komponente za svaki piksel na slici - R (red), G (green) i B (blue).
![images/cat_rgb.png](images/cat_rgb.png)

In [ ]:
img

Primetite da je svaki element matrice **uint8** (unsigned 8-bit integer), odnosno celobroja vrednost u intervalu [0, 255].

In [ ]:
img.dtype

### Osnovne operacije pomoću NumPy

Predstavljanje slike kao NumPy array je vrlo korisna stvar, jer omogućava jednostavnu manipulaciju i izvršavanje osnovih operacija nad slikom.

#### Isecanje (crop)

In [ ]:
img_crop = img[100:200, 300:600]  # prva koordinata je po visini (formalno red), druga po širini (formalo kolona)
plt.imshow(img_crop) # "najmanja vrednost" koordinate visine se nalazi na vrhu, a najveca na dnu

#### Okretanje (flip)

In [ ]:
img_flip_h = img[:, ::-1]  # prva koordinata ostaje ista, a kolone se uzimaju unazad
plt.imshow(img_flip_h)

In [ ]:
img_flip_v = img[::-1, :]  # druga koordinata ostaje ista, a redovi se uzimaju unazad
plt.imshow(img_flip_v)

In [ ]:
img_flip_c = img[:, :, ::-1]  # možemo i izmeniti redosled boja (RGB->BGR)
plt.imshow(img_flip_c)

#### Invertovanje

In [ ]:
img_inv = 255 - img  # ako su pikeli u intervalu [0,255] ovo je ok, a ako su u intervalu [0.,1.] onda bi bilo 1. - img
plt.imshow(img_inv)

### Konvertovanje iz RGB u "grayscale"

Konvertovanjem iz RGB modela u nijanse sivih (grayscale) se gubi informacija o boji piksela na slici, ali sama slika postaje mnogo lakša za dalju obradu.

Ovo se može uraditi na više načina:
1. **Srednja vrednost** RGB komponenti - najjednostavnija varijanta $$ G = \frac{R+G+B}{3} $$
2. **Metod osvetljenosti** - srednja vrednost najjače i najslabije boje $$ G = \frac{max(R,G,B) + min(R,G,B)}{2} $$
3. **Metod perceptivne osvetljenosti** - težinska srednja vrednost koja uzima u obzir ljudsku percepciju (npr. najviše smo osetljivi na zelenu boju, pa to treba uzeti u obzir)$$ G = 0.21*R + 0.72*G + 0.07*B $$

In [ ]:
# implementacija metode perceptivne osvetljenosti
def my_rgb2gray(img_rgb):
    img_gray = np.ndarray((img_rgb.shape[0], img_rgb.shape[1]))  # zauzimanje memorije za sliku (nema trece dimenzije)
    img_gray = 0.21*img_rgb[:, :, 0] + 0.72*img_rgb[:, :, 1] + 0.07*img_rgb[:, :, 2]
    img_gray = img_gray.astype('uint8')  # u prethodnom koraku smo mnozili sa float, pa sada moramo da vratimo u [0,255] opseg
    return img_gray

img_gray = my_rgb2gray(img)
plt.imshow(img_gray, 'gray') # kada se prikazuje slika koja nije RGB, obavezno je staviti 'gray' kao drugi parametar

Ipak je najbolje se držati implementacije u **OpenCV** biblioteci :).

In [ ]:
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [ ]:
img_gray.shape

In [ ]:
plt.imshow(img_gray, 'gray')  

In [ ]:
print(img_gray.shape)
print("Visina slike: ", img_gray.shape[0])
print("Sirina slike: ", img_gray.shape[1])
print("Broj komponenti: 1")

### Histogram

Histogram je grafički prikaz brojnosti piksela datog inteziteta. Daje nam informaciju o distribuciji osvetljenosti piksela.  

Pseudo-kod histograma za grayscale sliku:  
```code
inicijalizovati nula vektor od 256 elemenata  

za svaki piksel na slici:
    preuzeti inicijalni intezitet piksela
    uvecati za 1 broj piksela tog inteziteta

plotovati histogram
```

In [ ]:
def hist(image):
    height, width = image.shape[0:2]
    x = range(0, 256)
    y = np.zeros(256)
    
    for i in range(0, height):
        for j in range(0, width):
            pixel = image[i, j]
            y[pixel] += 1
    
    return (x, y)

x,y = hist(img_gray)
plt.plot(x, y, 'b')
plt.show()

Koristeći **matplotlib**:

In [ ]:
plt.hist(img_gray.ravel(), 255, [0, 255]) # .ravel() - prebacivanje visedimenzionalnog niza u jednodimenzionalni niz
plt.show()

Koristeći **OpenCV**:

In [ ]:
hist_full = cv2.calcHist([img_gray], [0], None, [255], [0, 255])
plt.plot(hist_full)
plt.show()

### Binarna slika

Slika čiji pikseli imaju samo dve moguće vrednosti: crno i belo. U zavisnosti da li interval realan (float32) ili celobrojan (uint8), ove vrednosti mogu biti {0,1} ili {0,255}.

U binarnoj slici često izdvajamo ono što nam je bitno (**foreground**), od ono što nam je nebitno (**background**). Formalnije, ovaj postupak izdvajanja bitnog od nebitnog na slici nazivamo **segmentacija**.

Najčešći način dobijanja binarne slike je korišćenje nekog praga (**threshold**), pa ako je vrednost piksela veća od zadatog praga taj piksel dobija vrednost 1, u suprotnom 0. Postoji više tipova threshold-ovanja:

1. Globalni threshold - isti prag se primenjuje na sve piksele
2. Lokalni threshold - različiti pragovi za različite delove slike
3. Adaptivni threshold - prag se ne određuje ručno (ne zadaje ga čovek), već kroz neki postupak. Može biti i globalni i lokalni.

#### Globalni threshold

Kako izdvojiti, na primer, samo lice?

In [ ]:
img_tr = img_gray > 127  # svi piskeli koji su veci od 127 ce dobiti vrednost True, tj. 1, i obrnuto
plt.imshow(img_tr, 'gray')

OpenCV ima metodu <b>threshold</b> koja:
* kao prvi parametar prima sliku koja se binarizuje
* kao drugi parametar prima prag binarizacije
* za treći parametar ima vrednost rezultujućeg piksela ako je veći od praga (255=belo)
* za poslednji parametar ima [tip thresholda](https://docs.opencv.org/4.4.0/d7/d1b/group__imgproc__misc.html#gaa9e58d2860d4afa658ef70a9b1115576) (u ovo slučaju je binarizacija).

In [ ]:
ret, image_bin = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY) # ret je vrednost praga, image_bin je binarna slika
print(ret)
plt.imshow(image_bin, 'gray')

#### Otsu threshold

[Otsu metoda](https://en.wikipedia.org/wiki/Otsu%27s_method) se koristi za automatsko pronalaženje praga za threshold na slici. Prag se pronalazi na osnovu histograma.  

Ideja: optimalan prag je obično u "dolinama" histograma.

In [ ]:
ret, image_bin = cv2.threshold(img_gray, 0, 255, cv2.THRESH_OTSU) # ret je izracunata vrednost praga, image_bin je binarna slika
print("Otsu's threshold: " + str(ret))
plt.imshow(image_bin, 'gray')

#### Adaptivni threshold

U nekim slučajevima primena globalnog praga za threshold ne daje dobre rezultate. Dobar primer su slike na kojima se menja osvetljenje, gde globalni threshold praktično uništi deo slike koji je previše osvetljen ili zatamnjen.

Adaptivni threshold je drugačiji pristup, gde se za svaki piksel na slici izračunava zaseban prag, na osnovu njemu okolnih piksela.  

In [ ]:
image_ada = cv2.imread('images/sonnet.png')
image_ada = cv2.cvtColor(image_ada, cv2.COLOR_BGR2GRAY)
plt.imshow(image_ada, 'gray')

In [ ]:
ret, image_ada_bin = cv2.threshold(image_ada, 100, 255, cv2.THRESH_BINARY)
plt.imshow(image_ada_bin, 'gray')

Loši rezultati su dobijeni upotrebom globalnog thresholda.  
Poboljšavamo rezultate korišćenjem adaptivnog thresholda. 

Pretposlednji parametar metode [adaptiveThreshold](https://docs.opencv.org/4.4.0/d7/d4d/tutorial_py_thresholding.html) je ključan, jer predstavlja veličinu bloka susednih piksela (npr. 15x15) na osnovnu kojih se računa lokalni prag.

Poslednji parametar predstavlja konstantnu vrednost, koja se oduzima od (ponderisane) srednje vrednosti izračunate za susedne piksele jednog bloka. Formula: $$T(x, y) = mean(pikseliBloka) - C$$ gde nam C predstavlja prethodno spomenutu konstantu.

In [ ]:
# adaptivni threshold gde se prag racuna = srednja vrednost okolnih piksela
image_ada_bin = cv2.adaptiveThreshold(image_ada, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, 5)
plt.figure() # ako je potrebno da se prikaze vise slika u jednoj celiji
plt.imshow(image_ada_bin, 'gray')

# adaptivni threshold gde se prag racuna = tezinska suma okolnih piksela, gde su tezine iz gausove raspodele
image_ada_bin = cv2.adaptiveThreshold(image_ada, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 5)
plt.figure()
plt.imshow(image_ada_bin, 'gray')

### Konverovanje iz grayscale/binarne u RGB

Ovo je zapravo trivijalna operacija koja za svaki kanal boje (RGB) napravi kopiju od originalne grayscale/binarne slike. Ovo je zgodno kada nešto što je urađeno u grayscale/binarnom modelu treba iskoristiti zajedno sa RGB slikom.

Npr. pretpostavimo da su vrednosti piksela lica između 100 i 200.

In [ ]:
img_tr = (img_gray > 100) * (img_gray < 200)
plt.imshow(img_tr, 'gray')

In [ ]:
img_tr_rgb = cv2.cvtColor(img_tr.astype('uint8'), cv2.COLOR_GRAY2RGB)

In [ ]:
plt.imshow(img*img_tr_rgb)  # množenje originalne RGB slike i slike sa izdvojenim pikselima lica

### Morfološke operacije

Veliki skup operacija za obradu digitalne slike, gde su te operacije zasnovane na oblicima, odnosno **strukturnim elementima**. U morfološkim operacijama, vrednost svakog piksela rezultujuće slike se zasniva na poređenju odgovarajućeg piksela na originalnoj slici sa svojom okolinom. Veličina i oblik ove okoline predstavljaju strukturni element.

In [ ]:
kernel = np.ones((3, 3)) # strukturni element 3x3 blok
print(kernel)

#### Erozija

Morfološka erozija postavlja vrednost piksela rez. slike na ```(i,j)``` koordinatama na **minimalnu** vrednost svih piksela u okolini ```(i,j)``` piksela na orig. slici.

U suštini erozija umanjuje regione belih piksela, a uvećava regione crnih piksela. Često se koristi za uklanjanje šuma (u vidu sitnih regiona belih piksela).

![images/erosion.gif](images/erosion.gif)

In [ ]:
plt.figure()
plt.imshow(cv2.erode(image_bin, kernel, iterations=1), 'gray')
plt.figure()
plt.imshow(cv2.erode(image_bin, kernel, iterations=3), 'gray')
plt.figure()
plt.imshow(cv2.erode(image_bin, kernel, iterations=5), 'gray')

#### Dilacija

Morfološka dilacija postavlja vrednost piksela rez. slike na ```(i,j)``` koordinatama na **maksimalnu** vrednost svih piksela u okolini ```(i,j)``` piksela na orig. slici.

U suštini dilacija uvećava regione belih piksela, a umanjuje regione crnih piksela. Zgodno za izražavanje regiona od interesa.

![images/dilation.gif](images/dilation.gif)

In [ ]:
# drugaciji strukturni element
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5)) # MORPH_ELIPSE, MORPH_RECT
print(kernel)

In [ ]:
plt.figure()
plt.imshow(cv2.dilate(image_bin, kernel, iterations=1), 'gray')
plt.figure()
plt.imshow(cv2.dilate(image_bin, kernel, iterations=3), 'gray')
plt.figure()
plt.imshow(cv2.dilate(image_bin, kernel, iterations=5), 'gray')

#### Otvaranje i zatvaranje

**```otvaranje = erozija + dilacija```**, uklanjanje šuma erozijom i vraćanje originalnog oblika dilacijom.

**```zatvaranje = dilacija + erozija```**, zatvaranje sitnih otvora među belim pikselima

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
print(kernel)
img_ero = cv2.erode(image_bin, kernel, iterations=1)
img_open = cv2.dilate(img_ero, kernel, iterations=1)
plt.imshow(img_open, 'gray')

In [ ]:
img_dil = cv2.dilate(image_bin, kernel, iterations=1)
img_close = cv2.erode(img_dil, kernel, iterations=1)
plt.imshow(img_close, 'gray')

**```detekcija ivica = dilacija - erozija (ili obrnuto)```**, Primer detekcije ivica na binarnoj slici korišćenjem dilacije i erozije:

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
image_edges = cv2.dilate(image_bin, kernel, iterations=1) - cv2.erode(image_bin, kernel, iterations=1)
plt.imshow(image_edges, 'gray')

OpenCV [alternativa](https://docs.opencv.org/4.4.0/d9/d61/tutorial_py_morphological_ops.html).

### Zamućenje (blur)

Zamućenje slike se dobija tako što se za svaki piksel slike kao nova vrednost uzima srednja vrednost okolnih piksela, recimo u okolini 5 x 5. Kernel <b>k</b> predstavlja kernel za <i>uniformno zamućenje</i>. Ovo je jednostavnija verzija <a href="https://en.wikipedia.org/wiki/Gaussian_blur">Gausovskog zamućenja</a>.

<img src="https://render.githubusercontent.com/render/math?math=k%285x5%29%3D%0A%20%20%5Cbegin%7Bbmatrix%7D%0A%20%20%20%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%5C%5C%0A%20%20%20%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%5C%5C%0A%20%20%20%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%5C%5C%0A%20%20%20%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%5C%5C%0A%20%20%20%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%20%26amp%3B%201%2F25%0A%20%20%5Cend%7Bbmatrix%7D&mode=display">

In [ ]:
from scipy import signal

k_size = 5
k = (1./k_size*k_size) * np.ones((k_size, k_size))
image_blur = signal.convolve2d(img_gray, k)
plt.imshow(image_blur, 'gray')

### Kontrast i osvetljenje slike

Kontrast i osvetljenje slike možemo podešavati pomoću metode <a href="https://www.tutorialspoint.com/how-to-change-the-contrast-and-brightness-of-an-image-using-opencv-in-python"><b>convertScaleAbs</b></a> iz openCV biblioteke.

Poziv metode izgleda ovako: <i><b>adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)</b></i>, a argumenti su:
* <b>image</b> predstavlja učitanu sliku pomoću cv2.imread metode
* <b>alpha</b> predstavlja vrednost kontrasta. Ako želimo da <b>smanjimo</b> kontrast, koristićemo vrednosti 0 < alpha < 1, a ako želimo da ga <b>povećamo</b>, koristićemo vrednosti > 1
* <b>beta</b> predstavlja vrednost osvetljenja. Preporučljiv opseg vrednosti je između [-127, 127]

In [ ]:
plt.imshow(img)

In [ ]:
alpha = 1.5
img_higher_contrast = cv2.convertScaleAbs(img, alpha=alpha)
plt.imshow(img_higher_contrast)

In [ ]:
alpha = 0.5
img_lower_contrast = cv2.convertScaleAbs(img, alpha=alpha)
plt.imshow(img_lower_contrast)

In [ ]:
beta = 20
img_higher_brightness = cv2.convertScaleAbs(img, beta=beta)
plt.imshow(img_higher_brightness)

In [ ]:
beta = -20
img_lower_brightness = cv2.convertScaleAbs(img, beta=beta)
plt.imshow(img_lower_brightness)

In [ ]:
alpha = 2
beta = 30
img_changed = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
plt.imshow(img_changed)

### Regioni i izdvajanje regiona

Najjednostavnije rečeno, region je skup međusobno povezanih belih piksela. Kada se kaže povezanih, misli se na to da se nalaze u neposrednoj okolini. Razlikuju se dve vrste povezanosti: tzv. **4-connectivity** i **8-connectivity**:

![images/48connectivity.png](images/48connectivity.png)

Postupak kojim se izdvajanju/obeležavaju regioni se naziva **connected components labelling**. Ovo ćemo primeniti na problemu izdvajanja barkoda.

In [ ]:
# ucitavanje slike i convert u RGB
img_barcode = cv2.cvtColor(cv2.imread('images/barcode.jpg'), cv2.COLOR_BGR2RGB)
plt.imshow(img_barcode)

Recimo da želimo da izdvojimo samo linije barkoda sa slike.

Za početak, uradimo neke standardne operacije, kao što je konvertovanje u grayscale i binarni threshold.

In [ ]:
img_barcode_gs = cv2.cvtColor(img_barcode, cv2.COLOR_RGB2GRAY) # konvert u grayscale
plt.imshow(img_barcode_gs, 'gray')

In [ ]:
ret, image_barcode_bin = cv2.threshold(img_barcode_gs, 80, 255, cv2.THRESH_BINARY)
plt.imshow(image_barcode_bin, 'gray')

Pokušaćemo i sa adaptivnim thresholdom.

In [ ]:
image_barcode_bin = cv2.adaptiveThreshold(img_barcode_gs, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 35, 10)
plt.imshow(image_barcode_bin, 'gray')

### Pronalaženje kontura/regiona

Konture, odnosno regioni na slici su grubo rečeno grupe crnih piksela. OpenCV metoda [**findContours**](https://docs.opencv.org/4.4.0/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0) pronalazi sve ove grupe crnih piksela, tj. regione. Prva povratna vrednost metode, odnosno *contours*, je lista pronađenih kontura na slici.

Ove konture je zaim moguće iscrtati metodom [**drawContours**](https://docs.opencv.org/4.4.0/d6/d6e/group__imgproc__draw.html#ga746c0625f1781f1ffc9056259103edbc), gde je: 
* prvi parametar slika na kojoj se iscrtavaju pronađene konture 
* drugi parametar lista kontura koje je potrebno iscrtati
* treći parametar određuje koju konturu po redosledu iscrtati (-1 znači iscrtavanje svih kontura)
* četvrti parametar je boja kojom će se obeležiti kontura
* poslednji parametar je debljina linije.

In [ ]:
contours, hierarchy = cv2.findContours(image_barcode_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

img = img_barcode.copy()
cv2.drawContours(img, contours, -1, (255, 0, 0), 1)
plt.imshow(img)

#### Osobine regiona

Svi pronađeni regioni imaju neke svoje karakteristične osobine: površina, obim, konveksni omotač, konveksnost, obuhvatajući pravougaonik, ugao...  

Ove osobine mogu biti izuzetno korisne kada je neophodno izdvojiti samo određene regione sa slike koji ispoljavaju neku osobinu.  

Za sve osobine pogledati [ovo](https://docs.opencv.org/4.4.0/dd/d49/tutorial_py_contour_features.html) i [ovo](https://docs.opencv.org/4.4.0/d1/d32/tutorial_py_contour_properties.html).

Izdvajamo samo bar-kod sa slike.

In [ ]:
contours_barcode = [] # ovde ce biti samo konture koje pripadaju bar-kodu

for contour in contours: # za svaku konturu
    center, size, angle = cv2.minAreaRect(contour) # pronadji pravougaonik minimalne povrsine koji ce obuhvatiti celu konturu
    height, width = size

    if width > 3 and width < 30 and height > 300 and height < 400: # uslov da kontura pripada bar-kodu
        contours_barcode.append(contour) # ova kontura pripada bar-kodu

img = img_barcode.copy()
cv2.drawContours(img, contours_barcode, -1, (255, 0, 0), 1)
plt.imshow(img)

In [ ]:
print('Ukupan broj regiona: %d' % len(contours_barcode))

Naravno, u ogromnom broj slučajeva odnos visine i širine neće biti dovoljan, već se moraju koristiti i ostale osobine (na primer, prethodni primer se može rešiti posmatranjem pozicije centara svih prisutnih kontura).

## Zadaci

1. Sa slike barkoda (**images/barcode.jpg**) izdvojiti samo brojeve i slova, bez linija barkoda.


2. Sa slike sa sijalicama (**images/bulbs.jpg**) prebrojati koliko ima sijalica.    


3. Na slici sa snouborderima (**images/snowboarders.jpg**) prebrojati koliko ima snoubordera.  


4. Na slici sa fudbalerima (**images/football.jpg**) izdvojiti samo one fudbalere u belim dresovima.  


5. Na slici sa crvenim krvnim zrncima (**images/bloodcells.jpg**), prebrojati koliko ima crvenih krvnih zrnaca.